In [192]:
from sqlalchemy import create_engine
from config import password
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd

# Extract

In [193]:
#create df from csv
services_df = pd.read_csv("db/311_Service_Requests__MiamiDade_County__2017.csv")

In [194]:
#preview csv
services_df.head()

,ticket_id,issue_type,issue_description,case_owner,case_owner_description,street_address,city,state,zip_code,neighborhood_district,...,location_city,sr_xcoordinate,sr_ycoordinate,latitude,longitude,method_received,goal_days,actual_completed_days,ticket_year,ObjectId
0,17-10002950,BULKY TRASH REQUEST,,Solid_Waste_Management,Solid Waste Management,10245 SW 37TH TER,Miami_Dade_County,Florida,33165.0,District 1,...,MIAMI,866560.749,509832.030,25.734851,-80.360870,PHONE,8,9.0,2017,2001
1,17-10002952,RECYCLING BLUE CART ISSUES,,Solid_Waste_Management,Solid Waste Management,15445 SW 114TH ST,Miami_Dade_County,Florida,33196.0,District 1,...,MIAMI,838824.626,482803.310,25.660831,-80.445467,PHONE,10,7.0,2017,2002
2,17-10002954,GREEN WASTE CART REQUEST,,Solid_Waste_Management,Solid Waste Management,14341 SW 151ST CT,Miami_Dade_County,Florida,33196.0,District 1,...,MIAMI,841126.314,472887.531,25.633522,-80.438608,PHONE,10,7.0,2017,2003
3,17-10002957,GREEN WASTE CART REQUEST,,Solid_Waste_Management,Solid Waste Management,12850 NW 8TH ST,Miami_Dade_County,Florida,33182.0,District 1,...,MIAMI,852183.120,524835.006,25.776314,-80.404338,PHONE,10,8.0,2017,2004
4,17-10002959,GARBAGE COMPLAINT,,Solid_Waste_Management,Solid Waste Management,3265 NW 46TH ST,Miami_Dade_County,Florida,33142.0,District 3,...,MIAMI,902895.391,539682.476,25.816450,-80.249984,PHONE,5,2.0,2017,2005


In [195]:
#check datatypes of columns
services_df.dtypes

ticket_id                         object
issue_type                        object
issue_description                 object
case_owner                        object
case_owner_description            object
street_address                    object
city                              object
state                             object
zip_code                         float64
neighborhood_district             object
created_year_month                 int64
ticket_created_date_time          object
ticket__last_update_date_time     object
ticket_closed_date_time           object
ticket_status                     object
location_city                     object
sr_xcoordinate                   float64
sr_ycoordinate                   float64
latitude                         float64
longitude                        float64
method_received                   object
goal_days                          int64
actual_completed_days            float64
ticket_year                        int64
ObjectId        

# Transform

In [196]:
#remove unnecessary columns
services_dropped_df = services_df.drop(columns=['ticket_id','issue_description','created_year_month','sr_xcoordinate','sr_ycoordinate','ticket_year', 'ObjectId'])

In [197]:
#preview df
services_dropped_df.head()

,issue_type,case_owner,case_owner_description,street_address,city,state,zip_code,neighborhood_district,ticket_created_date_time,ticket__last_update_date_time,ticket_closed_date_time,ticket_status,location_city,latitude,longitude,method_received,goal_days,actual_completed_days
0,BULKY TRASH REQUEST,Solid_Waste_Management,Solid Waste Management,10245 SW 37TH TER,Miami_Dade_County,Florida,33165.0,District 1,2017-01-04T05:00:00.000Z,2017-01-13T05:00:00.000Z,2017-01-13T05:00:00.000Z,CLOSED,MIAMI,25.734851,-80.360870,PHONE,8,9.0
1,RECYCLING BLUE CART ISSUES,Solid_Waste_Management,Solid Waste Management,15445 SW 114TH ST,Miami_Dade_County,Florida,33196.0,District 1,2017-01-04T05:00:00.000Z,2017-01-11T05:00:00.000Z,2017-01-11T05:00:00.000Z,CLOSED,MIAMI,25.660831,-80.445467,PHONE,10,7.0
2,GREEN WASTE CART REQUEST,Solid_Waste_Management,Solid Waste Management,14341 SW 151ST CT,Miami_Dade_County,Florida,33196.0,District 1,2017-01-04T05:00:00.000Z,2017-01-11T05:00:00.000Z,2017-01-11T05:00:00.000Z,CLOSED,MIAMI,25.633522,-80.438608,PHONE,10,7.0
3,GREEN WASTE CART REQUEST,Solid_Waste_Management,Solid Waste Management,12850 NW 8TH ST,Miami_Dade_County,Florida,33182.0,District 1,2017-01-04T05:00:00.000Z,2017-01-12T05:00:00.000Z,2017-01-12T05:00:00.000Z,CLOSED,MIAMI,25.776314,-80.404338,PHONE,10,8.0
4,GARBAGE COMPLAINT,Solid_Waste_Management,Solid Waste Management,3265 NW 46TH ST,Miami_Dade_County,Florida,33142.0,District 3,2017-01-04T05:00:00.000Z,2017-01-06T05:00:00.000Z,2017-01-06T05:00:00.000Z,CLOSED,MIAMI,25.816450,-80.249984,PHONE,5,2.0


In [ ]:
#check for incomplete tickets
services_dropped_df.loc[services_dropped_df['ticket_status'] != 'CLOSED'].head()
services_dropped_df.dtypes

# Load

In [ ]:
#Connect to local database
rds_connection_string = f"root:{password}@localhost/services_311_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
#Check tables
engine.table_names()

In [ ]:
#Load df into table in mysql
services_dropped_df.to_sql(name='requests_311', con=engine, if_exists='append', index=False)

In [ ]:
#Preview table to verify load worked
pd.read_sql_query('select * from requests_311', con=engine).head()